#### The goal of the project is to help the company generate incremental sales byimplementing a recommender system.
#### Data Prep and Modelling
▪ Missing data<br>
▪ Calculate find most popular product using maximum sales by volume and dollar amount<br>
▪ Company with maximum purchase<br>
▪ Build a popularity based recommender system<br>
▪ Build a recommender system using matrix factorization<br>
▪ Cosine similarity for product description

In [1]:
#import Libraries
import pandas as pd
pd.set_option("display.max_columns",200)
pd.set_option("display.max_rows",200)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("PBL 5 recommendation data.csv", encoding = "ISO-8859-1")

In [3]:
df.sample(3)

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,Customers.last_modified,Customers.customer_type,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.company,Orders.order_number,Orders.reorder_id,Orders.external_source,Orders.external_id,Orders.currency,Orders.sales_rep,Orders.subtotal,Orders.tax,Orders.shipping,Orders.coupon_id,Orders.coupon_amount,Orders.gift_id,Orders.gift_amount,Orders.fee_name,Orders.fee_amount,Orders.discount_name,Orders.discount_amount,Orders.total,Orders.balance_due,Orders.shipping_carrier,Orders.shipping_method,Orders.shipping_trans,Orders.shipping_flags,Orders.weight,Orders.tracking,Orders.payment_status,Orders.payment_date,Orders.payment_user,Orders.payment_type,Orders.payment_method,Orders.payment_amount,Orders.purchase_order,Orders.payment_id,Orders.payment_code,Orders.payment_ref,Orders.status,Orders.placed_date,Orders.updated_date,Orders.shipped_date,Orders.comments,Orders.notes,Orders.registry_id,Orders.gift_message,Orders.website,Orders.mailing,Orders.flags,Orders.partial_ship,Orders.customer_type,Order_Items.id,Order_Items.parent,Order_Items.product_id,Order_Items.product_name,Order_Items.attributes,Order_Items.attribute_names,Order_Items.attribute_prices,Order_Items.qty,Order_Items.price,Order_Items.cost,Order_Items.registry_item,Order_Items.related_id,Order_Items.reorder_frequency,Order_Items.account_id,Order_Items.flags,Products.id,Products.status,Products.product_type,Products.template,Products.vendor,Products.import_id,Products.name,Products.display_name,Products.menu_name,Products.list_price,Products.price,Products.sale_price,Products.cost,Products.flags,Products.left_flag,Products.right_flag,Products.last_modified,Products.taxable,Products.shopping_gtin,Products.shopping_brand,Products.shopping_mpn,Products.shopping_gender,Products.shopping_color,Products.shopping_age,Products.shopping_flags,Products.amazon_asin,Products.amazon_type,Products.amazon_item_type,Products.amazon_price,Products.google_shopping_id,Products.google_shopping_type,Products.google_shopping_cat,Products.google_adwords,Products.shopping_cat,Products.shopping_type,Products.pricegrabber_cat,Products.shopzilla_cat,Products.thefind_cat,Products.quickbooks_id,Products.qb_edit_sequence,Products.price_break_type,Products.price_breaks,Products.short_description,Products.long_description,Products.websites,Products.video,Products.audio,Products.seo_title,Products.seo_description,Products.seo_keywords,Products.seo_header,Products.seo_footer,Products.seo_url,Products.seo_category,Products.unit,Products.packaging,Products.display_packaging,Products.multiple,Products.length,Products.width,Products.height,Products.rx,Products.latex,Products.upc,Products.msds_link,Products.msds_label,Products.lit_link,Products.lit_label,Products.hcpcs,Products.case_qty,Products.markup,Products.override_markup,Products.notes,Products.import_flags,Products.map_price,Products.features_title,Products.warranty,Products.hygienic,Products.default_quantity,Products.user_size,Products.assembly,Products.installation,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.handling_time,Products.rotation_link,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
329,304,Alexander,Manly,NaN,1409081056,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1437763215,0.0,3613,304,Alexander,Manly,NaN,3613,NaN,NaN,NaN,USD,NaN,72.19,6.32,9.95,NaN,NaN,NaN,NaN,NaN,0.10,NaN,NaN,88.56,0.01,manual,0|Standard Shipping,NaN,NaN,NaN,1Z165YY60392434045,3.0,1.424874e+09,NaN,authorize.net,NaN,88.55,NaN,6952636249,56261W,NaN,1,1424873774,1.438868e+09,1.424966e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,51

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4194 entries, 0 to 4193
Columns: 181 entries, Customers.id to Products.freight_cost
dtypes: float64(98), int64(10), object(73)
memory usage: 5.8+ MB


In [5]:
df.isnull().sum()

Customers.id                         0
Customers.fname                      0
Customers.lname                      0
Customers.company                 3467
Customers.create_date                0
Customers.status                  4093
Customers.mailing                 2080
Customers.reminders               4194
Customers.tax_exempt              4189
Customers.account_id              4191
Customers.sales_rep               4194
Customers.rewards                 4194
Customers.profile_id              4194
Customers.last_modified              0
Customers.customer_type            423
Orders.id                            0
Orders.customer_id                   0
Orders.fname                         0
Orders.lname                         0
Orders.company                    3466
Orders.order_number                  0
Orders.reorder_id                 4112
Orders.external_source            3829
Orders.external_id                4189
Orders.currency                      0
Orders.sales_rep         

### Data Prep and Modelling
▪ **Missing data**

In [6]:
df.shape

(4194, 181)

First remove empty columns

In [7]:
df2=df.dropna(axis=1,how="any")

Then add back important columns

In [8]:
df2["Customers.company"]=df["Customers.company"]
df2["Products.shopping_brand"]=df["Products.shopping_brand"]
df2["Orders.payment_method"]=df["Orders.payment_method"]
df2["Orders.shipping_method"]=df["Orders.shipping_method"]
df=df2 #it is simpler to just work with 'df'
df.sample(3)

,Customers.id,Customers.fname,Customers.lname,Customers.create_date,Customers.last_modified,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.order_number,Orders.currency,Orders.subtotal,Orders.total,Orders.status,Orders.placed_date,Order_Items.id,Order_Items.parent,Order_Items.product_name,Order_Items.qty,Order_Items.price,Customers.company,Products.shopping_brand,Orders.payment_method,Orders.shipping_method
1770,1472,Steven,Devine,1440450069,1456953337,5701,1472,Steven,Devine,5701,USD,57.19,67.14,1,1440450190,7432,5701,Half Rail for Medline Homecare Beds,1,57.19,NaN,MEDLINE,NaN,11|Ground
2865,2433,William,Gallagher,1454689545,1456953454,12467,2433,William,Gallagher,12467,USD,68.78,78.73,1,1454689698,14804,12467,Galvanic Skin Response - GSR 2 biofeedback unit,1,68.78,NaN,Fabrication Enterprises Inc.,NaN,0|Standard Shipping
135,134,Virginia,Lopez,1399929511,1437763190,560,134,Virginia,Lopez,560,USD,51.20,61.15,1,1399929512,921,560,ReadyBath Rinse-Free Shampoo and Conditioning ...,1,51.20,NaN,MEDLINE,NaN,11|Ground


Get the full name of customer and delete the first and last name columns

In [9]:
df["Customers.name"]=df["Customers.fname"]+" "+df["Customers.lname"]
df.drop(["Customers.fname","Customers.lname"],axis=1,inplace=True)
df.sample(3)

,Customers.id,Customers.create_date,Customers.last_modified,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.order_number,Orders.currency,Orders.subtotal,Orders.total,Orders.status,Orders.placed_date,Order_Items.id,Order_Items.parent,Order_Items.product_name,Order_Items.qty,Order_Items.price,Customers.company,Products.shopping_brand,Orders.payment_method,Orders.shipping_method,Customers.name
1261,1043,1431801710,1437559425,4258,1043,Lee,Ceccotti,4258,USD,129.35,126.36,1,1431801744,5847,4258,FitRight Active Male Guards,1,65.0,NaN,MEDLINE,NaN,11|Ground,Lee Ceccotti
557,467,1416844151,1437763150,2305,467,Peter,lashenka,2305,USD,144.10,154.05,1,1416844152,3760,2305,WCATR012,1,28.0,NaN,NaN,NaN,0|Standard Shipping,Peter lashenka
1430,1186,1435007122,1437559241,4620,1186,Jacklyn,Anderson,4620,USD,172.50,172.50,1,1435007246,6256,4620,"Medline Excel K3 Leightweight Wheechair, Black...",1,172.5,NaN,MEDLINE,NaN,11|Ground,Jacklyn Anderson


**▪ Calculate find most popular product using maximum sales by volume and dollar amount<br>**

In [15]:
df['Order_Items.product_name'].nunique()

1821

In [35]:
products = df.groupby(['Order_Items.product_name','Order_Items.qty']).sum()[['Order_Items.price']]
products.head()

,,Order_Items.price
Order_Items.product_name,Order_Items.qty,
"BUCKET, REPLACEMENT FOR MDS89668XW",1,44.00
1 Box Of Medline Commode Liners with Absorbent Pad,2,18.99
1 Pad of CURAD Mediplast Wart Pad,1,4.88
"1-Ply Blockade AngelStat Surgical Gowns, Ceil Blue, Extra Large",1,235.05
"1/2"" self-adhesive hook material, 25 yard dispenser box, white",1,23.25


In [55]:
df['Order_Itmes.total_amount']=df['Order_Items.qty']*df['Order_Items.price']
products = df.groupby(['Order_Items.product_name','Order_Items.qty','Order_Items.price']).sum()[['Order_Itmes.total_amount']]
products.head(3)

,,,Order_Itmes.total_amount
Order_Items.product_name,Order_Items.qty,Order_Items.price,
"BUCKET, REPLACEMENT FOR MDS89668XW",1,44.00,44.00
1 Box Of Medline Commode Liners with Absorbent Pad,2,18.99,37.98
1 Pad of CURAD Mediplast Wart Pad,1,4.88,4.88


In [54]:
#Product with most total amount
popproducts.sort_values(by='Order_Itmes.total_amount',ascending=False).head(3)

,,,Order_Itmes.total_amount
Order_Items.product_name,Order_Items.qty,Order_Items.price,
"Disposable Emergency Blanket, Gray, Not Applicable",30,34.61,8306.40
Sanitary Pads with Adhesive & Wings,122,42.32,5163.04
"2-Ply Tissue/Poly Professional Towels, White, Not Applicable",200,25.27,5054.00


Here we can see that the most dolar amount was of <font color='darkblue'>**_Disposable Emergency Blanket_**</font> with an amount of <font color='darkred'>_8306.40€_</font> <br>
There is also a quite large difference to the second product with _5163.04$_

In [53]:
#Product with most quantitiy
mostqty = df.groupby(['Order_Items.product_name','Order_Items.qty']).sum()[['Order_Items.price']]
mostqty.sort_values(by='Order_Items.qty',ascending=False).head(3)

,,Order_Items.price
Order_Items.product_name,Order_Items.qty,
TruTemp,500,2.99
"2-Ply Tissue/Poly Professional Towels, White, Not Applicable",200,25.27
Sanitary Pads with Adhesive & Wings,122,42.32


Here we can see that the product with most Quantity is <font color='darkblue'>**_TruTemp_**</font> with <font color='darkred'>_500_</font>

In [52]:
#Product with highest price
mostprc = df.groupby(['Order_Items.product_name','Order_Items.price']).sum()[['Order_Items.qty']]
mostprc.sort_values(by='Order_Items.price',ascending=False).head(3)

,,Order_Items.qty
Order_Items.product_name,Order_Items.price,
Cirrus Plus EC Folding Power Wheelchair,1899.0,1
"Battery Powered Electric Patient Lift with Rechargeable and Removable Battery, With Wall Mount",1899.0,1
"Titan Front Wheel Power Wheelchair 20"" Captain Seat",1475.0,1


Here we can see that the product with the highest price is <font color='darkblue'>**_Cirrus Plus EC Folding Power Wheelchair_**</font> with <font color='darkred'>_1899$_</font>

**▪ Company with maximum purchase**

In [56]:
#company with maximum purchase
maxcomp = products = df.groupby('Customers.company').sum()[['Order_Itmes.total_amount']]
maxcomp.sort_values(by='Order_Itmes.total_amount',ascending=False).head(3)

,Order_Itmes.total_amount
Customers.company,
Company59,13186.41
Company343,5628.56
Company17,4167.15


Here we can see that the company with the maximum purchase is <font color='darkblue'>**_Company59_**</font> with <font color='darkred'>_13186.41_$</font>

**▪ Build a popularity based recommender system<br>
▪ Build a recommender system using matrix factorization<br>**

In [62]:
grouped=df.groupby('Order_Items.product_name').agg({ "Order_Items.qty":[np.size,np.sum,np.mean],"Order_Items.price":[np.mean]})
grouped["Order_Itmes.total_amount"]=grouped["Order_Items.qty"]["sum"]*grouped["Order_Items.price"]["mean"]
grouped.head(3)

Order_Items.qty           \
                                                              size sum mean   
Order_Items.product_name                                                      
 BUCKET, REPLACEMENT FOR MDS89668XW                              1   1  1.0   
1 Box Of Medline Commode Liners with Absorbent Pad               1   2  2.0   
1 Pad of CURAD Mediplast Wart Pad                                1   1  1.0   

                                                   Order_Items.price  \
                                                                mean   
Order_Items.product_name                                               
 BUCKET, REPLACEMENT FOR MDS89668XW                            44.00   
1 Box Of Medline Commode Liners with Absorbent Pad             18.99   
1 Pad of CURAD Mediplast Wart Pad                               4.88   

                                                   Order_Itmes.total_amount  
                                                                             
Order_Items.product_name                                                     
 BUCKET, REPLACEMENT FOR MDS89668XW                                   44.00  
1 Box Of Medline Commode Liners with Absorbent Pad                    37.98  
1 Pad of CURAD Mediplast Wart Pad                                      4.88

In [69]:
grouped=df.groupby("Order_Items.product_name").agg({"Order_Itmes.total_amount":[np.size,np.sum,np.mean]})
grouped.head()

Order_Itmes.total_amount  \
                                                                       size   
Order_Items.product_name                                                      
 BUCKET, REPLACEMENT FOR MDS89668XW                                       1   
1 Box Of Medline Commode Liners with Absorbent Pad                        1   
1 Pad of CURAD Mediplast Wart Pad                                         1   
1-Ply Blockade AngelStat Surgical Gowns, Ceil B...                        1   
1/2" self-adhesive hook material, 25 yard dispe...                        1   

                                                                    
                                                       sum    mean  
Order_Items.product_name                                            
 BUCKET, REPLACEMENT FOR MDS89668XW                  44.00   44.00  
1 Box Of Medline Commode Liners with Absorbent Pad   37.98   37.98  
1 Pad of CURAD Mediplast Wart Pad                     4.88    4.88  
1-Ply Blockade AngelStat Surgical Gowns, Ceil B...  235.05  235.05  
1/2" self-adhesive hook material, 25 yard dispe...   23.25   23.25

In [72]:
populer=grouped.sort_values(("Order_Itmes.total_amount","mean"),ascending=False)
populer.head()

Order_Itmes.total_amount  \
                                                                       size   
Order_Items.product_name                                                      
Baseline&reg; Dynamometer - Smedley Spring - Ad...                        1   
Sanitary Pads with Adhesive & Wings                                       2   
2-Ply Tissue/Poly Professional Towels, White, N...                        2   
Battery Powered Electric Patient Lift with Rech...                        1   
Cirrus Plus EC Folding Power Wheelchair                                   1   

                                                                      
                                                        sum     mean  
Order_Items.product_name                                              
Baseline&reg; Dynamometer - Smedley Spring - Ad...  3000.00  3000.00  
Sanitary Pads with Adhesive & Wings                 5628.56  2814.28  
2-Ply Tissue/Poly Professional Towels, White, N...  5559.40  2779.70  
Battery Powered Electric Patient Lift with Rech...  1899.00  1899.00  
Cirrus Plus EC Folding Power Wheelchair             1899.00  1899.00

In [73]:
toplam=grouped["Order_Itmes.total_amount"]["sum"].sum()
toplam # this is going to help for the percentage calculation

391519.3

In [75]:
populer["percentage"]=populer["Order_Itmes.total_amount","sum"].div(toplam)*100
populer.head()

Order_Itmes.total_amount  \
                                                                       size   
Order_Items.product_name                                                      
Baseline&reg; Dynamometer - Smedley Spring - Ad...                        1   
Sanitary Pads with Adhesive & Wings                                       2   
2-Ply Tissue/Poly Professional Towels, White, N...                        2   
Battery Powered Electric Patient Lift with Rech...                        1   
Cirrus Plus EC Folding Power Wheelchair                                   1   

                                                                      \
                                                        sum     mean   
Order_Items.product_name                                               
Baseline&reg; Dynamometer - Smedley Spring - Ad...  3000.00  3000.00   
Sanitary Pads with Adhesive & Wings                 5628.56  2814.28   
2-Ply Tissue/Poly Professional Towels, White, N...  5559.40  2779.70   
Battery Powered Electric Patient Lift with Rech...  1899.00  1899.00   
Cirrus Plus EC Folding Power Wheelchair             1899.00  1899.00   

                                                   percentage  
                                                               
Order_Items.product_name                                       
Baseline&reg; Dynamometer - Smedley Spring - Ad...   0.766246  
Sanitary Pads with Adhesive & Wings                  1.437620  
2-Ply Tissue/Poly Professional Towels, White, N...   1.419956  
Battery Powered Electric Patient Lift with Rech...   0.485034  
Cirrus Plus EC Folding Power Wheelchair              0.485034

In [76]:
populer.sort_values(("percentage"),ascending=False)

Order_Itmes.total_amount  \
                                                                       size   
Order_Items.product_name                                                      
Disposable Emergency Blanket, Gray, Not Applicable                       14   
Medline MoliCare Super Plus Disposable Adult Di...                       53   
Medline Deluxe Aluminum Transport Chair With Ha...                       46   
Sanitary Pads with Adhesive & Wings                                       2   
2-Ply Tissue/Poly Professional Towels, White, N...                        2   
...                                                                     ...   
OR Sterile Specimen Containers, 120.0 ML                                  1   
1 Pad of CURAD Mediplast Wart Pad                                         1   
Male External Catheters, Medium                                           1   
Sterile and Non-Sterile Pediatric Urine Collect...                        1   
Rectangular Plastic Washbasins, Graphite, 8.000                           1   

                                                                           \
                                                         sum         mean   
Order_Items.product_name                                                    
Disposable Emergency Blanket, Gray, Not Applicable  10556.05   754.003571   
Medline MoliCare Super Plus Disposable Adult Di...   8984.40   169.516981   
Medline Deluxe Aluminum Transport Chair With Ha...   5940.54   129.142174   
Sanitary Pads with Adhesive & Wings                  5628.56  2814.280000   
2-Ply Tissue/Poly Professional Towels, White, N...   5559.40  2779.700000   
...                                                      ...          ...   
OR Sterile Specimen Containers, 120.0 ML                5.68     5.680000   
1 Pad of CURAD Mediplast Wart Pad                       4.88     4.880000   
Male External Catheters, Medium                         4.24     4.240000   
Sterile and Non-Sterile Pediatric Urine Collect...      3.16     3.160000   
Rectangular Plastic Washbasins, Graphite, 8.000         2.60     2.600000   

                                                   percentage  
                                                               
Order_Items.product_name                                       
Disposable Emergency Blanket, Gray, Not Applicable   2.696176  
Medline MoliCare Super Plus Disposable Adult Di...   2.294753  
Medline Deluxe Aluminum Transport Chair With Ha...   1.517305  
Sanitary Pads with Adhesive & Wings                  1.437620  
2-Ply Tissue/Poly Professional Towels, White, N...   1.419956  
...                                                       ...  
OR Sterile Specimen Containers, 120.0 ML             0.001451  
1 Pad of CURAD Mediplast Wart Pad                    0.001246  
Male External Catheters, Medium                      0.001083  
Sterile and Non-Sterile Pediatric Urine Collect...   0.000807  
Rectangular Plastic Washbasins, Graphite, 8.000      0.000664  

[1821 rows x 4 columns]

If our customer never bought any products, we should recommend the <font color='darkblue'>**_Disposable Emergency Blanket, Gray, Not Applicable_**</font>

#### Matrix Factorization
- we make a column for every product
- then we see how much they bought of each product

In [77]:
prod_matrix=df.pivot_table(index="Orders.customer_id",columns='Order_Items.product_name',values='Order_Items.qty').fillna(0)
prod_matrix.head(3)

Order_Items.product_name,"BUCKET, REPLACEMENT FOR MDS89668XW",1 Box Of Medline Commode Liners with Absorbent Pad,1 Pad of CURAD Mediplast Wart Pad,"1-Ply Blockade AngelStat Surgical Gowns, Ceil Blue, Extra Large","1/2"" self-adhesive hook material, 25 yard dispenser box, white","1/2"" self-adhesive loop material, 25 yard dispenser box, white","100% Cotton Unisex Reversible Hyperbaric Scrub Tops, Misty Green, MZS","100% Cotton Unisex Reversible Tops, Ceil Blue, MHS",10215-s,12112-3,12124-3,"18"" Hamper Stands, Standard","19"" Flyweight Lightweight Blue Transport Wheelchair","2"" self-adhesive loop material, 10 yard dispenser box, tan","2"" self-adhesive loop material, 25 yard dispenser box, tan","2-Ply Tissue/Poly Professional Towels, White, Not Applicable",2000 Extra-Wide Wheelchairs,2000 Wheelchairs,"24 Hour Urine Collection Bottle, 3000.0 ML","3 Pack, Hospital Receiving Blankets, Baby Blankets, 100% Cotton, 30x40, Elephant","3 Pack, Hospital Receiving Blankets, Baby Blankets, 100% Cotton, 34x40, Footprints",3 Wheel Flame Blue Rollator Walker with Basket Tray and Pouch,3 Wheel Flame Red Rollator Walker with Basket Tray and Pouch,"3 Wheel Walker Rollator with Basket Tray and Pouch, Flame Blue","3 Wheel Walker Rollator with Basket Tray and Pouch, Flame Red","3"" Wheels For Walkers","3/4"" Guardian Utility Tips, Gray","4-Panel Abdominal Binders, 2X-Large","4-Panel Abdominal Binders, Large/X-Large","5"" Wheels For Walkers","6 Cylinder Oxygen Carrier, M9 Cylinders",6' CPAP Tube,6-spring hand grip (each),6291hda Universal wheels part,"8"" Rear Walker Wheel Attachment","A&D Ointment by H & H Laboratories Inc, 384.00 ML",Accu-Therm Hot/Cold Gel Packs,Accu-Therm Insulated Hot Pack,Accu-Therm Non-Insulated Hot Pack,"Accutouch Chemo Nitrile Exam Gloves, Blue, Medium","Active Liquid Protein Nutritional Supplement, 0.0","Active Liquid Protein Nutritional Supplement, 30.000 OZ",AcuForce&reg; 7.0 Massage Tool,Adaptive Eating Utensils - Adaptive Fork,Addipak Sterile Saline Solution,Adjustable Angle Rotating Suction Cup Grab Bar,Adjustable Height ArmRest AK5ADJARMKIT,Adjustable Height Bathtub Grab Bar Safety Rail,Adjustable Height Home Bed Assist Handle,"Adjustable Height Rollator with 6"" Wheels, Red","Adjustable Height Teak Bath Bench Stool, Rectangular","Adjustable Height Teak Bath Bench Stool, Triangular",Adjustable Height Triangular Teak Bath Bench Stool,"Adjustable Lightweight Folding Cane with Gel Hand Grip, Black","Adjustable Lightweight Folding Cane with Gel Hand Grip, Blue Crackle","Adjustable Lightweight Folding Cane with Gel Hand Grip, Red Crackle",Adult Bariatric Crutches,"AdvanTrode&reg; Elite Electrode, 2"" round, tan tricot, 40/box",Advantage Magnetic Patient Alarms,Advantage Select SE Mattresses,Advantage Therapeutic Homecare Foam Mattress,Aero-Pulse Pressure Pads,Aeroflow II Wheelchair Cushions,Aeroflow Static Air Overlay,Aeromist Plus Nebulizer Compressor with Disposable Nebulizer Kit,Airone Alternating Pressure Pads,All Weather Premium Quality Shopping Utility Bag - Green,"Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves, Large","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves, Medium","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves, Small","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves, X-Large","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves, Green, Medium","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves, Green, Small","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves, Green, X-Large",Aloetouch PROTECT Dimethicone Skin Protectant Wipes,Aloetouch Personal Cleansing Wipes,"Aloetouch Powder-Free Latex Exam Gloves, Green, X-Small",Aloetouch SELECT Premium Spunlace Personal Cleansing Wipes,Aloetouch Sensitive Personal Cleansing Baby Wipes,"Aloetouch Ultra IC Powder-Free Latex-Free Synthetic Exam Gloves, Large","Aloetouch Ultra IC Powder-Free Latex-Free Synthetic Exam Gloves, Medium",Aluminum Bath Benches with Back,Aluminum Folding Commode with Padded Armrests,"Aluminum Forearm 

In [123]:
testproduct = prod_matrix['Disposable Emergency Blanket, Gray, Not Applicable']
prod_matrix.corrwith(testproduct).sort_values(ascending=False).to_frame()

,0
Order_Items.product_name,
"Disposable Emergency Blanket, Gray, Not Applicable",1.000000
"Disposable Emergency Blanket, Not Applicable",0.830068
"Disposable Polypropylene Fitted Stretcher Sheets, Blue, Not Applicable",0.054738
Suresite Window Transparent Dressings,-0.000545
Push-Button Aluminum Crutches,-0.000545
...,...
"Medline Emesis/Barf Bags, Blue - 24 Pack",-0.002597
Protection Plus Disposable Underpads,-0.002623
K1 Basic Extra-Wide Wheelchairs,-0.003160


In [121]:
testproduct2 = prod_matrix['TruTemp']
prod_matrix.corrwith(testproduct2).sort_values(ascending=False).to_frame()

,0
Order_Items.product_name,
TruTemp,1.000000
"Phlebotomy Sharps Containers, Red, 1.000 QT",0.707028
"Protection Plus Overnight Protective Underwear, 40.00",-0.000328
"Enteral Feeding and Irrigation Syringes, 60.00",-0.000328
Medline MSC327100 Capri Bladder Control Pads,-0.000328
...,...
"Medline Emesis/Barf Bags, Blue - 24 Pack",-0.001561
Protection Plus Disposable Underpads,-0.001577
K1 Basic Extra-Wide Wheelchairs,-0.001900


In [124]:
testproduct3 = prod_matrix['Sanitary Pads with Adhesive & Wings']
prod_matrix.corrwith(testproduct3).sort_values(ascending=False).to_frame()

,0
Order_Items.product_name,
Sanitary Pads with Adhesive & Wings,1.000000
Suresite Window Transparent Dressings,-0.000328
"FitRight Active Male Guards, 6"" X 11""",-0.000328
Caring Non-Sterile Abdominal Pads,-0.000328
"Ladies' Resistat Lab Coats, White, White",-0.000328
...,...
"Medline Emesis/Barf Bags, Blue - 24 Pack",-0.001561
Protection Plus Disposable Underpads,-0.001577
K1 Basic Extra-Wide Wheelchairs,-0.001900


**▪ Cosine similarity for product description**

In [129]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [127]:
vect = CountVectorizer()
sparse_matrix=vect.fit_transform(df)

In [128]:
matrix = sparse_matrix.todense()
df = pd.DataFrame(matrix,columns=vect.get_feature_names())
df

,company,create_date,currency,customer_id,customers,fname,id,last_modified,lname,name,order_items,order_itmes,order_number,orders,parent,payment_method,placed_date,price,product_name,products,qty,shipping_method,shopping_brand,status,subtotal,sum,total,total_amount
0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [130]:
pd.DataFrame(cosine_similarity(df,df))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,1.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0
1,0.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0
2,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0
3,0.5,0.0,0.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
4,0.0,0.0,0.0,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
5,0.0,0.0,0.0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
6,0.0,0.0,0.0,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
7,0.0,0.0,0.0,0.5,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
8,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0
9,0.0,0.0,0.0,0.5,0.5,0.5,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0


#### K-Nearest Neighbor Based Recommender System
We will take the cosine distances of the products and create a vector of the product

In [102]:
#sort the products from largest to smallest according to their quantity.
productProperties = df.groupby("Order_Items.product_name").agg({"Order_Items.qty": [np.size, np.mean]})
productProperties.head(5)

Order_Items.qty     
                                                              size mean
Order_Items.product_name                                               
 BUCKET, REPLACEMENT FOR MDS89668XW                              1  1.0
1 Box Of Medline Commode Liners with Absorbent Pad               1  2.0
1 Pad of CURAD Mediplast Wart Pad                                1  1.0
1-Ply Blockade AngelStat Surgical Gowns, Ceil B...               1  1.0
1/2" self-adhesive hook material, 25 yard dispe...               1  1.0

In [108]:
prodTotAmount = pd.DataFrame(productProperties["Order_Items.qty"]["size"])
prodNormalizedTotAmount = prodTotAmount.apply(lambda x: (x-np.min(x)) / (np.max(x) - np.min(x)))
prodNormalizedTotAmount.sample(5)

,size
Order_Items.product_name,
"Drive Medical Aluminum Lightweight Transport Chair, 19"", Blue",0.038462
"Med Aire Alternating Pressure Pump and Pad System, Variable Pressure",0.096154
Vinyl Innerspring Mattresses,0.000000
Universal Cup Holder,0.057692
"Transfer Board, Wood, 8"" x 30"", two handgrips",0.000000


How often did a product get bought and whats its mean value, and their percentage                                     

In [112]:
#This is where I tried to run the KNN Based code from our lessons, but I didn't fully understand the code and couldn't implement it here

In [111]:
productDict = {}

with open(r'PBL 5 recommendation data.csv', encoding="iso-8859-1") as f:
    
    for line in f:
        fields = line.rstrip('\n').split('|')
        prodID = df["Order_Items.id"] # prodId'ye aliyoruz, 0. sutuna eslestiriyoruz
        name = df["Order_Items.product_name"] #ismini buraya aliyoruz
        qty = df["Order_Items.qty"] # quantity
        qty = list(map(int, qty)) # bunu list yapiyoruz
        productDict[prodID] = (name, qty, prodNormalizedTotAmount.loc[prodID].get('size'), productProperties.loc[prodID].df.get('mean'))
        # burada ürünlere ayit bir sözlük olusturduk

KeyError: "None of [Int64Index([ 5284,    31, 11655,  1816,    40,   341,  2997,  1797,  1796,\n             1795,\n            ...\n            20163, 20166, 20167, 20168, 20182, 20186, 20190, 20191, 20196,\n            20199],\n           dtype='int64', name='Order_Items.product_name', length=4194)] are in the [index]"

In [ ]:
#bir ürünün diger ürüne uzakligini bulan function
from scipy import spatial

def ComputeDistance(a, b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA, genresB) # cosinus uzakligini buluyoruz, bir genre'nin diger genre'ya
    popularityA = a[2]
    popularityB = b[2]
    popularitiyDistance = abs(popularityA - popularityB) # bir popularity'nin diger popularity'e uzakligini ve mutlak degerini aliyoruz ( abs() )
    return genreDistance + popularitiyDistance

In [ ]:
#aradaki mesafeler en kücük olanlari liste olarak istiyoruz
import operator

def getNeighbors(prodID, K):
    distances = []
    for product in productDict:
        if (product != prodID):
            dist = ComputeDistance(productDict[prodID], productDict[product])
            distances.append((product, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K=30
avgQuantity = 0
neighbors = getNeighbors(1, K)
for neighbor in neighbors:
    avgQuantity += productDict[neighbor][3]
    print( productDict[neighbor][0] + " " + str(productDict[neighbor][3]))